# Lab03: TF-IDF.

- MSSV: 19120689
- Họ và tên: Lại Khánh Toàn

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; từ `TODO` cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `tf_idf_data.txt` của các bạn (nếu file này kích thước lớn các bạn có thể chép link vào `link_data.txt`), nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Cài đặt một web crawler để thu thập và biểu diễn dữ liệu bằng không gian vector và trọng số TF-IDF: https://en.wikipedia.org/wiki/Web_mining. Truy vấn dựa trên cosine similarity

## Nội dung bài tập

## 1. Không gian vector

- Một vector 2 chiều có thể được viết dưới dạng `[x,y]`
- Một vector 3 chiều có thể được viết dưới dạng `[x,y,z]`
![vector](vector.png)

- Đặt $V$ là tập hợp các đặc trưng trong thể hiện dữ liệu.
- Bất kỳ mẫu dữ liệu nào đều có thể được biểu diễn dưới dạng một vectơ với $\vert V\vert$ chiều
- Ví dụ: giả sử chúng ta có 3 đặc trưng là các từ dog, bite, man. Giá trị 1 thể hiện từ đó xuất hiện 1 lần, 0 là không xuất hiện.  
![space](space.png)


## 2. Thu thập và thể hiện dữ liệu
Thu thập dữ liệu bắt đầu từ URL: https://en.wikipedia.org/wiki/Web_mining.
- Gọi $D$ là một tập văn bản chứa *n* văn bản: $D=\left\{d_1,d_2,...,d_n\right\}$.
- $V=\left\{v_1,v_2,...v_{\vert V \vert}\right\}$ là từ điển (tất cả các từ phân biệt trong dữ liệu thu được). $\vert V \vert$ là kích thước từ điển.
- Một trọng số $w_{ij}$ được gán cho từ $t_i$ của văn bản dj thuộc $D$ xác định mức quan trọng của $t_i$ trong văn bản $d_j$. Từ không xuất hiện trong $d_j$ có $w_{ij}=0$.
- Mỗi văn bản $d_j$ được thể hiện dưới dạng vector $\mathbf{d_j}= [w_{1j},w_{2j},...,w_{\vert V \vert j}]$
- Thể hiện dữ liệu bằng một ma trận M kích thước $n \times \vert V \vert$ => mỗi hàng thể hiện một văn bản.

In [1]:
#Cài đặt Lab 2
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string
import pickle

import random
import urllib.robotparser

import nltk
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# from num2words import num2words

In [3]:
def get_urls(url):
    r = requests.get(url)
    # TODO
    # Lấy các url nằm trong trang web của url này, lưu lại vào biến urls
    soup = BeautifulSoup(r.text, 'html.parser')
    urls = []
    for link in soup.find_all('a', href= re.compile('http[s]?://(?:[-\w.])+')): 
      urls.append(link.get('href'))

    return urls

def get_urls_recursive(start_url, limit):
    urls = [start_url]
    
        # TODO
        # Lấy các url nằm trong trang web của url này, lưu lại vào biến new_urls
        # Với mỗi url mới trong new_urls:
        #   Nếu nó chưa nằm trong urls thì thêm nó vô  
        # Nếu kích thước của urls vượt quá limit thì dừng và xóa phần dư thừa
    
    for url in urls:
      
      
      new_url = get_urls(url)

      for val in new_url:
        if val not in urls:
          urls.append(val)
      if len(urls) > limit:
        urls = urls[:limit]
        break
    return urls


In [4]:
# limit = 200
limit = 60
url_list = get_urls_recursive('https://en.wikipedia.org/wiki/Web_mining', limit)

In [5]:
def text_filter(element):
    # TODO
    # Cài đặt lại như Lab02
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

def wordList(url):
    # TODO
    # Cài đặt lại như Lab02
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    text = soup.findAll(text=True)
    filtered_text = list(filter(text_filter, text)) # list của các chuỗi
    word_list = []

    for strg in filtered_text: 
      new_strg = strg.replace(string.punctuation, " ")
      words = new_strg.split()
      word_list.extend(words)

    return word_list

def read_url(url, url_idx, data):
    # TODO
    # Cài đặt lại như Lab02
    word_list = wordList(url)

    for word in word_list:
      if word not in data:
        data[word] = [[url_idx], 1]
      else:
        if url_idx not in data[word][0]:
          data[word][0].append(url_idx)
        data[word][1]+=1

In [6]:
data = {}
for url_index, url in enumerate(url_list, 1):
    read_url(url, url_index, data)

## 3. Tiền xử lý dữ liệu

In [7]:
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
print(english_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# TODO
# Loại bỏ các key của biến data mà nằm trong danh sách english_stopwords
for val in list(data):
  if val in english_stopwords:
    del data[val]

In [9]:
# TODO
# chuyển đổi dữ liệu về chữ thường và gộp các key trùng
data = {key.lower():val for key, val in data.items()}


In [15]:
dict1 = {'a': [1, 2], 'b': [3, 4], 'b': [5, 6]}

dict2 = {}

for key, val in dict1.items():
    print(r'{0} : {1}'.format(key, val))
    # if key not in dict2:
    #     dict2[key] = val
    # else:
    #     dict2[key].extend(val)

print(dict2)

a : [1, 2]
b : [5, 6]
{}


In [ ]:

# merge duplicate keys
for key in list(data):
    data[key.lower()][0].extend(data[key][0])
    data[key.lower()][1] += data[key][1]
    del data[key]
    

In [10]:
# merger các key trùng
for key in list(data):
    if key.isnumeric():
        data['number'] = data.get('number', [[], 0])
        data['number'][0].extend(data[key][0])
        data['number'][1] += data[key][1]
        del data[key]
data

{'main': [[1, 11, 26, 31, 32, 43, 44, 53, 55], 22],
 'menu': [[16, 18, 23, 24, 39, 40, 50], 12],
 'move': [[1, 26, 30, 31, 32, 34, 43, 44, 53, 55], 29],
 'sidebar': [[1, 26, 30, 31, 32, 34, 43, 44, 53, 55], 29],
 'hide': [[1, 31, 34, 43, 44, 55], 17],
 'page': [[1, 44], 4],
 'contents': [[1, 20, 23, 26, 30, 31, 32, 41, 43, 44, 50, 53, 54, 55], 14],
 'current': [[1], 1],
 'events': [[1], 1],
 'random': [[18], 1],
 'article': [[1, 8, 18], 9],
 'about': [[1, 44], 2],
 'wikipedia': [[1, 2, 11, 16, 23, 32, 35, 37, 38, 41, 43, 44, 50, 52, 58], 34],
 'contact': [[2, 35], 3],
 'us': [[1, 2, 44], 3],
 'donate': [[2], 2],
 'help': [[53], 1],
 'learn': [[1, 31, 33, 34, 43, 44, 53, 55], 8],
 'edit': [[1, 44], 4],
 'community': [[1], 1],
 'portal': [[8, 15, 23, 32, 39, 40, 46], 7],
 'recent': [[1, 2], 2],
 'changes': [[1, 2, 44], 5],
 'upload': [[1, 11, 44], 4],
 'file': [[1, 44], 3],
 'languages': [[1, 31, 43, 44, 55], 5],
 'language': [[1, 22, 26, 27, 31, 34, 43, 44, 48, 49, 53, 55], 15],
 'links

In [ ]:
# TODO
# loại bỏ các từ không phải tiếng anh
for key in data:
    if not key.isascii():
        del data[key]

In [ ]:
#TODO
# chuyển đổi các từ về dạng nguyên gốc và gộp các key trùng
# VD: created => create, creating => create
# Gợi ý: sử dụng nltk.stem và PorterStemmer
ps = PorterStemmer()
for key in list(data):
    if key not in data:
        continue
    data[ps.stem(key)] = data.pop(key)

# TODO
# chuyển đổi các số về dạng chữ
# VD: 123 => one two three
# Gợi ý: sử dụng num2words
for key in list(data):
    if key not in data:
        continue
    if key.isdigit():
        data[num2words(key)] = data.pop(key)

## 4. TF-IDF (Term Frequency - Inverse Document Frequency)
TF-IDF cho biết độ quan trọng của một từ đối với một tài liệu trong ngữ liệu.

TF- Term Frequency : dùng để ước lượng tần xuất xuất hiện của từ trong văn bản. Tuy nhiên với mỗi văn bản thì có độ dài khác nhau số lần xuất hiện của từ có thể nhiều hơn . Vì vậy số lần xuất hiện của từ sẽ được chuẩn hóa bằng cách chia cho độ dài của văn bản (tổng số từ trong văn bản đó).
    
$$tf_{t}=\dfrac{f(t,d)}{\sum_{i \in d}f(i,d)}$$ 

- $f(t,d)$ là số lần xuất hiện từ $t$ trong văn bản $d$.

IDF- Inverse Document Frequency: Khi tính tần số xuất hiện TF thì các từ đều được coi là quan trọng như nhau. Tuy nhiên có một số từ thường được được sử dụng nhiều nhưng không quan trọng để thể hiện ý nghĩa của đoạn văn như "is", "the"... (các từ chức năng). Ta cần giảm độ quan trọng của những từ này.

$$idf_t=\log \left(\dfrac{n}{df_t}\right)$$

- *n* là số văn bản.

- $df_t$ là số văn bản xuất hiện từ t

**TF-IDF:** $$tf_{t} \times idf_t$$

- $tf_{t}$ càng lớn tần suất xuất hiện của từ trong văn bản càng lớn.
- $idf_t$ càng lớn từ hiếm khi xuất hiện trong tập dữ liệu.
- **Giả định** những đặc trưng quan trọng nhất là những đặc trưng hiếm xuất hiện.

In [ ]:
#TODO
def cal_tf_idf(data, n_doc):
    # n_words = ?
    M = np.zeros(shape=(n_doc,n_words),dtype=float)
   

    return M

In [ ]:
#TODO
# n_urls=200

M = cal_tf_idf(data, n_urls)

#TODO save data M to txt file using numpy


## 4. Truy vấn thông tin

### Công thức tính Cosine Similarity

Tính toán độ tương tự của truy vấn 𝐪 với
mỗi tài liệu $𝐝_𝑗$ trong tập hợp tài liệu 𝐷

![cosine](cosine.PNG)  

In [ ]:
#TODO
def cal_cosine_similarity(d, q):
    #tính toán giá trị Cosine similarity với từng tài liệu dj trong tập hợp tài liệu D 
    
    return cosine_similarity

In [ ]:
def information_retrieval(q):
    # q: query
    # trả về đường dẫn có trang chứa giá trị Cosine similarity cao nhất với câu query
    cosine_similarities = []
    for w in M:
        cosine_similarities.append(cal_cosine_similarity(w, q))
    
    #TODO


    return {"url": link, "cosine_similarity": cosine_similarity}

In [ ]:
print(information_retrieval("Web mining"))